In [2]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from peft import LoraConfig


import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
Gemma2ForCausalLM
)
from trl import SFTTrainer
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator
import peft
from langchain.llms import HuggingFacePipeline

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
# #QLoRA
# lora_config = LoraConfig(
#     r=6,#멀티헤드어텐션 헤드 개수
#     lora_alpha = 8, #어텐션 계수 스케일
#     lora_dropout = 0.05, #드롭아웃 비율
#     target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
#     task_type="CAUSAL_LM",
# )

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

In [ ]:
model_id = "google/gemma-2-2b-it"

os.environ["HF_TOKEN"] = ""

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
    token=os.environ["HF_TOKEN"]
)



# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)
tokenizer.use_default_system_prompt = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1
)

# ✅ 3. LangChain LLM wrapper
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
<ipython-input-16-306505157c25>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [21]:
# 벡터 임베딩 및 검색
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

### 데이터 부르기

In [4]:
import zipfile

with zipfile.ZipFile('open (1).zip', 'r') as zip_ref:
    zip_ref.extractall('data')


In [28]:
file_path = 'data/test.csv'
test_data = pd.read_csv(file_path)

df = test_data.sample(frac=1).reset_index(drop=True)

df['context'][0]

"“아이를 너무 많이 낳는다”며 나라가 출산아 제한에 나선다면? 인구증가를 막기 위해 국가 차원에서 ‘가족계획’을 강조한 나라가 있다면? 먼 나라 이야기라고 생각할지 모르지만, 실제 우리나라에서 일어났던 일이다. 우리 정부는 1983년 산아제한과 ‘가족계획’의 필요성을 담은 영화를 제작해 이를 대대적으로 홍보했다.\n\n 51초에 한명이 태어나는 1980년대엔 인구증가를 억제하는 것이 나라의 몫이었다. 이 때문에 정부는 ‘문화영화’를 별도로 제작해 1998년 6월까지 영화관에서 틀게 했다. ‘국민 계몽’을 위한다는 명목에서였다. 80년대 당시 가족계획과 관련한 유명한 표어는 ‘잘 키운 딸 하나, 열 아들 안 부럽다’ 였다. 그나마 '덮어놓고 낳다 보면 거지꼴을 못 면한다'(60년대), '아들, 딸 구별 말고 둘만 낳아 잘 기르자'(70년대)에 비해 업그레이드된 버전이다.\n\n 정책 홍보와 국민 계몽을 목적으로 정부가 만든 ‘문화영화’가 이번에 공개됐다. 국가기록원은 10일 ‘문화영화로 보는 대한민국’을 주제로 정부가 제작했던 1980년대부터 1990년대 사이의 문화영화 227편을 공개했다. 당시 시대상을 한눈에 볼 수 있는 자료로 국가기록원 홈페이지를 통해 11일부터 볼 수 있다.\n\n 1982년에 만들어진 문화영화 ‘인플레를 잡는 길’엔 당시 폭등했던 부동산 이야기가 담겨있다. “한탕주의 대표적인 예가 76년부터 78년도 사이에 전국을 휩쓸었던 부동산 투기”라며 “불과 1년 사이 집값은 무려 2배나 뛰어올랐고, 심지어 아파트는 몇백만원씩 프리미엄이 붙은 과열 투기로 번졌다”는 것이다.\n\n 당시 정부는 “인플레를 잡지 못하면 경제 성장이 어렵게 되고, 자연히 일자리가 부족해져 많은 실업자가 생기게 된다”는 설명과 함께 저축을 장려했다.\n\n 문화영화는 다양한 주제를 다뤘다. 1987년엔 ‘민주화의 길’을 제목으로 “우리는 폭력적 방법에 의한 혁명을 하려는 것이 아니고 선거에 의한 개혁을 통해 민주주의를 확립해 나가려 하고 있다”고 알렸다.\n\n 19

In [29]:
df

,id,context,question
0,TEST_1094,“아이를 너무 많이 낳는다”며 나라가 출산아 제한에 나선다면? 인구증가를 막기 위해...,가족계획과 관련된 1980년대 표어는?
1,TEST_0716,발생지역인 중국 연안에서 우리나라에 떠밀려와 제주도와 전남 해안 일대의 김 양식에 ...,괭생이모자반이 대량 발생하는 기간은
2,TEST_0702,제주특별자치도는 6일부터 특수형태 근로종사자와 프리랜서를 대상으로 한 제3차 긴급고...,영업제한 업종 지원금액
3,TEST_1332,조 바이든이 미국의 차기 대통령이 되면 한국 수출에 날개를 달아줄 수 있다는 기대가...,한국 일평균 수출이 플러스가 된 건 몇 개월 만인가
4,TEST_0318,지난해 제주관광공사 매출액이 2019년 대비 42.3% 줄고 1억7400만원의 영업...,제주관광공사의 2020년도 매출액은
...,...,...,...
1502,TEST_1063,4일 오전 강원 화천군 7사단 신병교육대. 6주간의 신병교육을 마친 훈련병들이 10...,신병 수료식에 참석한 훈련병 수
1503,TEST_0121,삼성전자는 7일부터 13일까지 ‘갤럭시 노트 20’ 사전 예약을 진행한다. 공식 출...,갤럭시 노트20의 공식 출시 날짜는 언제인가요
1504,TEST_0910,빌 게이츠 전 마이크로소프트(MS) 최고경영자(CEO)가 8일(현지시간) 미국 매체...,트럼프 대통령이 틱톡 관련 서명한 날짜는
1505,TEST_0251,미국 대선이 바이든 민주당 후보의 승리로 마무리 단계로 접어들면서 미국의 이민정책에...,미국투자이민 설명회가 개최되는 장소는 어디인가


In [ ]:
generated_answers = []
true_answers = []

prompt_template = """<start_of_turn>user
다음은 뉴스 기사입니다. 기사를 잘 읽고, 질문에 단답형으로 정확하게 답해주세요.

기사:
{context}

질문:
{question}

답변은 한두 단어로 간결하게 해주세요.
<end_of_turn>
<start_of_turn>model
"""


for idx, row in df.iterrows():
    article = row["context"]
    question = row["question"]
    #answer = row["answer"]

    # 문서 쪼개기 & 벡터화
    docs = text_splitter.create_documents([article])
    vectordb = FAISS.from_documents(docs, embedding_model)

    # QA 체인 구성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(),
        return_source_documents=False
    )

    # 프롬프트 생성
    prompt = prompt_template.format(context=article, question=question)


    try:
        response = qa_chain({"query": prompt})
        generated = response["result"].strip()
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        generated = ""

    generated_answers.append(generated)
    true_answers.append(answer.strip())

correct = 0
total = len(true_answers)

# for pred, true in zip(generated_answers, true_answers):
#     if pred.strip() == true.strip():
#         correct += 1

# accuracy = correct / total
# print(f"\n전체 문항 수: {total}")
# print(f"맞춘갯수: {correct}")
# print(f" Exact Match Accuracy: {round(accuracy * 100, 2)}%")

In [ ]:
df = pd.DataFrame(list(df.items()), columns=['id', 'answer'])
df.to_csv( 'data/submission_pure_rag.csv', index=False, encoding='utf-8-sig')